# EXTRACTION OF CONTEXT FROM IMAGE 
(OPTIONAL)

In [1]:
# !sudo apt install tesseract-ocr
# !pip install pytesseract

In [2]:
# import pytesseract
# import shutil
# import os
# import random
# try:
# from PIL import Image
#except ImportError:
# import Image

In [3]:
# from google.colab import files
# uploaded = files.upload()

In [4]:
# image_path_in_colab=‘image.jpg’
# extractedInfo = pytesseract.image_to_string(
#                                     Image.open(image_path_in_colab))
# print(extractedInfo)
# extractedInfo = context

# SUMMARIZATION WITH Transformers by Hugging Face

In [5]:
!pip install keyphrase-vectorizers

!pip install transformers
import torch

from transformers import AutoTokenizer, AutoModelWithLMHead

tokenizer = AutoTokenizer.from_pretrained('t5-base')
model = AutoModelWithLMHead.from_pretrained('t5-base', return_dict=True)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1322: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language

In [6]:
import nltk
nltk.download('punkt')
nltk.download('brown')
nltk.download('wordnet')
from nltk.corpus import wordnet as wn
from nltk.tokenize import sent_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [7]:
def summarize(text,model,tokenizer):
  inputs = tokenizer.encode("summarize: " + text, 
                            return_tensors='pt', 
                            max_length=512,
                            truncation=True)
  ids = model.generate(inputs, 
                               max_length=500, 
                               min_length=80, 
                               length_penalty=5., 
                               num_beams=2)


  summary = tokenizer.decode(ids[0])
  processed=""
  for sent in sent_tokenize(summary):
    sent = sent.capitalize()
    processed = processed +" "+sent
  processed.capitalize()
  processed = processed.strip()
  return processed


# NOUN PHRASE EXTRACTION

In [8]:
!pip install git+https://github.com/boudinfl/pke.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/boudinfl/pke.git to /tmp/pip-req-build-xbwqd0in
  Running command git clone --filter=blob:none --quiet https://github.com/boudinfl/pke.git /tmp/pip-req-build-xbwqd0in
  Resolved https://github.com/boudinfl/pke.git to commit ebd6e5754b4156a61a4ec6c4c283e821d11a36be
  Preparing metadata (setup.py) ... done


In [9]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import string
import pke
import traceback

def get_nounphrases(content):
    out=[]
    try:
        extract= pke.unsupervised.MultipartiteRank()
        extract.load_document(input=content,language='en')
        #    not contain punctuation marks or stopwords as candidates.
        pos = {'PROPN','NOUN'}
        #pos = {'PROPN','NOUN'}
        stoplist = list(string.punctuation)
        stoplist += ['-lrb-', '-rrb-', '-lcb-', '-rcb-', '-lsb-', '-rsb-']
        stoplist += stopwords.words('english')
        extract.candidate_selection(pos=pos)
    
        extract.candidate_weighting(alpha=1.1,
                                      threshold=0.75,
                                      method='average')
        keyphrases = extract.get_n_best(n=10)
        

        for val in keyphrases:
            out.append(val[0])
    except:
        out = []
        traceback.print_exc()

    return out

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# KEY WORD EXTRACTION using KEYBERT

In [10]:
!pip install keybert
from keybert import KeyBERT



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [11]:
from keyphrase_vectorizers import KeyphraseCountVectorizer
vectorizer = KeyphraseCountVectorizer()

In [12]:
kw_model = KeyBERT()

# Question Qeneration with Transformers by Huggingface

---



In [13]:
!pip install transformers[sentencepiece]


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [14]:
from transformers import AutoModelWithLMHead, AutoTokenizer

model_name = "mrm8488/t5-base-finetuned-question-generation-ap"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelWithLMHead.from_pretrained(model_name)

def get_question(answer, context, max_length=64):
  input_text = "answer: %s  context: %s </s>" % (answer, context)
  features = tokenizer([input_text], return_tensors='pt')

  output = model.generate(input_ids=features['input_ids'], 
               attention_mask=features['attention_mask'],
               max_length=max_length)

  quest = tokenizer.decode(output[0])
  quest = quest.strip()
  return quest




The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.


# 2ND Answer generation by Transformers Pipeline

In [15]:
from transformers import pipeline

model_checkpoint = "huggingface-course/bert-finetuned-squad"

In [16]:
import pprint

# THE FINAL c0de !

In [17]:
def get_mca_questions(context: str):
  if type(context)==str:
    summarized = summarize(context,model,tokenizer)
    imp_words= get_nounphrases(summarized)
    ot_keywords = kw_model.extract_keywords(
                                context,
                                vectorizer=vectorizer,
                                use_mmr = True)
    st_keywords = kw_model.extract_keywords(
                                summarized,
                                vectorizer=vectorizer,
                                use_mmr = True)
    otk=[]
    for i in ot_keywords:
      o = list(i)
      for i in o:
        if type(i)==str:
          otk.append(i)
    for i in st_keywords:
      s = list(i)
      for i in s:
        if type(i)==str:
          otk.append(i)
          
    imp_words = list(imp_words)
    for i in otk:
      imp_words.append(i)
    imp_words = set(imp_words)
    imp_words = list(imp_words)
    imp_words = sorted(imp_words)

    qn_list = []
    ans_list = []
    for index , answer in zip(range(10),imp_words):
      ques = get_question(answer,summarized)
      qn_list.append(ques)
      ans_list.append(answer.capitalize())
    ans2_list = []

    qa_model = pipeline("question-answering",model_checkpoint)
    for q in qn_list:
      ans = qa_model(question = q, context = summarized)
      ans2_list.append(ans.get('answer'))
   
    ans2_list = [i.title() for i in ans2_list]

    option_2 = get_nounphrases(summarized) 
    option_3= get_nounphrases(context)
    option_3 = [i.title() for i in option_3]
  
    import random
    o4 = get_nounphrases(summarized)
    option_4 = random.sample(o4, len(o4))
    option_4 = [i.title() for i in option_4]

    f = []
    for i,j,k,l,m in zip(qn_list,ans_list,ans2_list,option_3,option_4):
      n = {}
      n['*']= i
      n['1'] = j
      n['2'] = k
      n['3'] = l
      n['4'] = m

      f.append(n)
  
    mca_questions = pprint.pprint(f)
  
    return mca_questions
  else:
    print("Please input the context in the form of paragraph(i.e string type)")

# CREDITS/CITATIONS


*) Inspired by Questgen.ai

*) Thanks and Credits to
   Tutorial by Ramsrigoutham for MCQ questions generation from text.

*) @misc{mromero2021t5-base-finetuned-question-generation-ap,
  title={T5 (base) fine-tuned on SQUAD for QG via AP},
  author={Romero, Manuel},
  publisher={Hugging Face},
  journal={Hugging Face Hub},
  howpublished={\url{https://huggingface.co/mrm8488/t5-base-finetuned-question-generation-ap}},
  year={2021}
}

*) https://github.com/MaartenGr/keyBERT




